In [1]:
import pandas as pd
import unicodedata
import re
from difflib import get_close_matches

In [2]:
stats_2024 = pd.read_csv('../general/stats/2024_batting_stats_edited.csv')
stats_2025 = pd.read_csv('../general/stats/2025_batting_stats_edited.csv')
def_rank_2024 = pd.read_csv('def_rank_hits_allowed_pitchers_2024.csv')
def_rank_2025 = pd.read_csv('../scrapers/def_scraper/batter_hits_def_rank.csv')
batter_team_2024 = pd.read_csv('../general/stats/2024/batters_team_2024.csv')
batter_team_2025 = pd.read_csv('../general/stats/batter_team_2025.csv')

In [3]:
#dropping first half of the 2024 season
# stats_2024 = stats_2024.iloc[len(stats_2024)//2:]

In [4]:
stats_2024['games_played'] = 1
stats_2025['games_played'] = 1

In [5]:
stats_2025

,Unnamed: 0,away_hitters,away_AB,away_R,away_H,away_RBI,away_HR,away_BB,away_K,away_AVG,...,home_K,home_AVG,home_OBP,home_SLG,away_player,away_team,home_player,home_team,gameID,games_played
0,0,F. LindorSS,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.293,...,0.0,0.257,0.357,0.544,Francisco Lindor,NYM,Ben Rice,NYY,1,1
1,1,J. SotoRF,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.255,...,1.0,0.409,0.495,0.766,Juan Soto,NYM,Aaron Judge,NYY,1,1
2,2,P. Alonso1B,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.305,...,1.0,0.238,0.304,0.397,Pete Alonso,NYM,Cody Bellinger,NYY,1,1
3,3,B. NimmoLF,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.230,...,1.0,0.345,0.393,0.500,Brandon Nimmo,NYM,Paul Goldschmidt,NYY,1,1
4,4,M. VientosDH,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.243,...,0.0,0.240,0.336,0.426,Mark Vientos,NYM,Jasson Dominguez,NYY,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,7608,L. Urias2B,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.259,...,0.0,0.216,0.263,0.443,Luis Urias,ATH,Ryan O'Hearn,BAL,630,1
7609,7609,N. Kurtz1B,5.0,2.0,2.0,1.0,1.0,0.0,1.0,0.281,...,1.0,0.172,0.303,0.287,Nick Kurtz,ATH,Michael Conforto,LAD,630,1
7610,7610,team,43.0,11.0,18.0,11.0,5.0,1.0,11.0,NaN,...,0.0,0.250,0.274,0.333,NaN,ATH,Miguel Rojas,LAD,630,1
7611,7611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,0.219,0.219,0.281,NaN,ATH,Chris Taylor,LAD,630,1


In [6]:
away_teams_2024=stats_2024[['away_player','away_H','home_team','games_played','away_AB','away_BB','away_K']]
away_teams_2025=stats_2025[['away_player','away_H','home_team','games_played','away_AB','away_BB','away_K']]

In [7]:
home_teams_2024=stats_2024[['home_player','home_H','away_team','games_played','home_AB','home_BB','home_K']]
home_teams_2025=stats_2025[['home_player','home_H','away_team','games_played','home_AB','home_BB','home_K']]

In [8]:
away_teams_2024.dropna()
away_teams_2025.dropna()
home_teams_2024.dropna()
home_teams_2025.dropna()

,home_player,home_H,away_team,games_played,home_AB,home_BB,home_K
0,Ben Rice,1.0,NYM,1,2.0,0.0,0.0
1,Aaron Judge,0.0,NYM,1,2.0,0.0,1.0
2,Cody Bellinger,0.0,NYM,1,2.0,0.0,1.0
3,Paul Goldschmidt,0.0,NYM,1,2.0,0.0,1.0
4,Jasson Dominguez,1.0,NYM,1,2.0,0.0,0.0
...,...,...,...,...,...,...,...
7607,Andy Pages,0.0,ATH,1,4.0,0.0,1.0
7608,Ryan O'Hearn,1.0,ATH,1,4.0,0.0,0.0
7609,Michael Conforto,3.0,ATH,1,4.0,0.0,1.0
7610,Miguel Rojas,0.0,ATH,1,4.0,0.0,0.0


In [9]:
# Convert both columns to numeric, coercing errors to NaN
away_teams_2024[['away_AB','away_BB']] = away_teams_2024[['away_AB','away_BB']].apply(
    pd.to_numeric, errors='coerce'
)
# Convert both columns to numeric, coercing errors to NaN
away_teams_2025[['away_AB','away_BB']] = away_teams_2025[['away_AB','away_BB']].apply(
    pd.to_numeric, errors='coerce'
)
# Convert both columns to numeric, coercing errors to NaN
home_teams_2024[['home_AB','home_BB']] = home_teams_2024[['home_AB','home_BB']].apply(
    pd.to_numeric, errors='coerce'
)
# Convert both columns to numeric, coercing errors to NaN
home_teams_2025[['home_AB','home_BB']] = home_teams_2025[['home_AB','home_BB']].apply(
    pd.to_numeric, errors='coerce'
)

C:\Users\Joshua\AppData\Local\Temp\ipykernel_113148\605779913.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_teams_2024[['away_AB','away_BB']] = away_teams_2024[['away_AB','away_BB']].apply(
C:\Users\Joshua\AppData\Local\Temp\ipykernel_113148\605779913.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_teams_2025[['away_AB','away_BB']] = away_teams_2025[['away_AB','away_BB']].apply(
C:\Users\Joshua\AppData\Local\Temp\ipykernel_113148\605779913.py:10: SettingWithCopyWarning: 
A value is tryin

In [10]:
home_teams_2025

,home_player,home_H,away_team,games_played,home_AB,home_BB,home_K
0,Ben Rice,1.0,NYM,1,2.0,0.0,0.0
1,Aaron Judge,0.0,NYM,1,2.0,0.0,1.0
2,Cody Bellinger,0.0,NYM,1,2.0,0.0,1.0
3,Paul Goldschmidt,0.0,NYM,1,2.0,0.0,1.0
4,Jasson Dominguez,1.0,NYM,1,2.0,0.0,0.0
...,...,...,...,...,...,...,...
7608,Ryan O'Hearn,1.0,ATH,1,4.0,0.0,0.0
7609,Michael Conforto,3.0,ATH,1,4.0,0.0,1.0
7610,Miguel Rojas,0.0,ATH,1,4.0,0.0,0.0
7611,Chris Taylor,1.0,ATH,1,4.0,0.0,2.0


In [11]:
# require at least 2 plate appearances (AB + BB >= 2)
away_teams_2024 = away_teams_2024[
    (away_teams_2024['away_AB'] + away_teams_2024['away_BB']) >= 2
].reset_index(drop=True)
away_teams_2025 = away_teams_2025[
    (away_teams_2025['away_AB'] + away_teams_2025['away_BB']) >= 2
].reset_index(drop=True)

In [12]:
# Version 1: require at least 3 plate appearances (AB + BB >= 3)
home_teams_2024 = home_teams_2024[
    (home_teams_2024['home_AB'] + home_teams_2024['home_BB']) >= 2
].reset_index(drop=True)
home_teams_2025 = home_teams_2025[
    (home_teams_2025['home_AB'] + home_teams_2025['home_BB']) >= 2
].reset_index(drop=True)


In [13]:
away_teams_2024

,away_player,away_H,home_team,games_played,away_AB,away_BB,away_K
0,Francisco Lindor,0.0,NYY,1,2.0,1.0,0.0
1,Juan Soto,1.0,NYY,1,2.0,1.0,1.0
2,Pete Alonso,1.0,CHC,1,3.0,0.0,0.0
3,Brandon Nimmo,0.0,STL,1,2.0,1.0,1.0
4,Mark Vientos,0.0,NYY,1,2.0,0.0,1.0
...,...,...,...,...,...,...,...
12027,Hunter Renfroe,1.0,ATL,1,4.0,0.0,2.0
12028,Freddy Fermin,1.0,ATL,1,3.0,0.0,1.0
12029,Maikel Garcia,0.0,ATL,1,2.0,1.0,1.0
12030,Kyle Isbel,0.0,ATL,1,2.0,1.0,2.0


In [14]:
away_teams_2024=stats_2024[['away_player','away_H','home_team','games_played']]
away_teams_2025=stats_2025[['away_player','away_H','home_team','games_played']]

In [15]:
home_teams_2024=stats_2024[['home_player','home_H','away_team','games_played']]
home_teams_2025=stats_2025[['home_player','home_H','away_team','games_played']]

In [16]:
team_mapping = {
    'Milwaukee Brewers': 'MIL',
    'Minnesota Twins': 'MIN',
    'Chicago Cubs': 'CHC',
    'Colorado Rockies': 'COL',
    'Washington Nationals': 'WSH',
    'Toronto Blue Jays': 'TOR',
    'Atlanta Braves': 'ATL',
    'Baltimore Orioles': 'BAL',
    'New York Yankees': 'NYY',
    'Philadelphia Phillies': 'PHI',
    'Pittsburgh Pirates': 'PIT',
    'Kansas City Royals': 'KCR',
    'Arizona Diamondbacks': 'ARI',
    'Boston Red Sox': 'BOS',
    'Detroit Tigers': 'DET',
    'Cleveland Guardians': 'CLE',
    'Tampa Bay Rays': 'TBR',
    'San Francisco Giants': 'SFG',
    'Los Angeles Dodgers': 'LAD',
    'Seattle Mariners': 'SEA',
    'Cincinnati Reds': 'CIN',
    'Texas Rangers': 'TEX',
    'Chicago White Sox': 'CHW',
    'Los Angeles Angels': 'LAA',
    'Miami Marlins': 'MIA',
    'New York Mets': 'NYM',
    'Athletics': 'ATH',
    'St. Louis Cardinals': 'STL',
    'San Diego Padres': 'SDP',
    'Houston Astros': 'HOU'
}

def_rank_2024['team'] = def_rank_2024['team'].map(team_mapping)
def_rank_2025['Team'] = def_rank_2025['Team'].map(team_mapping)

In [17]:
#merge df_rank_2024 into merged_df_2 to bring in the def_rank 
merged_df_away_2024 = away_teams_2024.merge(
    def_rank_2024,
    how='left',
    left_on='home_team',
    right_on='team'
)
merged_df_away_2024 = merged_df_away_2024.drop(columns=['team'])

In [18]:
merged_df_away_2024=merged_df_away_2024[['away_player','away_H','home_team','Rank','games_played']]

In [19]:
merged_df_away_2024

,away_player,away_H,home_team,Rank,games_played
0,Francisco Lindor,0.0,NYY,6.0,1
1,Juan Soto,1.0,NYY,6.0,1
2,Pete Alonso,1.0,CHC,14.0,1
3,Brandon Nimmo,0.0,STL,23.0,1
4,Mark Vientos,0.0,NYY,6.0,1
...,...,...,...,...,...
14260,Maikel Garcia,0.0,ATL,8.0,1
14261,Kyle Isbel,0.0,ATL,8.0,1
14262,NaN,5.0,ATL,8.0,1
14263,NaN,NaN,ATL,8.0,1


In [20]:
#merge df_rank_2024 into merged_df_2 to bring in the def_rank 
merged_df_home_2024 = home_teams_2024.merge(
    def_rank_2024,
    how='left',
    left_on='away_team',
    right_on='team'
)
merged_df_home_2024 = merged_df_home_2024.drop(columns=['team'])

In [21]:
merged_df_home_2024=merged_df_home_2024[['home_player','home_H','away_team','Rank','games_played']]

In [22]:
#merge df_rank_2025 into merged_df_2 to bring in the def_rank 
merged_df_away_2025 = away_teams_2025.merge(
    def_rank_2025,
    how='left',
    left_on='home_team',
    right_on='Team'
)
merged_df_away_2025 = merged_df_away_2025.drop(columns=['Team'])

In [23]:
merged_df_away_2025=merged_df_away_2025[['away_player','away_H','home_team','Rank','games_played']]

In [24]:
#merge df_rank_2024 into merged_df_2 to bring in the def_rank 
merged_df_home_2025 = home_teams_2025.merge(
    def_rank_2025,
    how='left',
    left_on='away_team',
    right_on='Team'
)
merged_df_home_2025 = merged_df_home_2025.drop(columns=['Team'])

In [25]:
merged_df_home_2025=merged_df_home_2025[['home_player','home_H','away_team','Rank','games_played']]

In [26]:
merged_df_home_2024 = merged_df_home_2024.dropna(subset=['home_player'])
merged_df_away_2024 = merged_df_away_2024.dropna(subset=['away_player'])
merged_df_home_2025 = merged_df_home_2025.dropna(subset=['home_player'])
merged_df_away_2025 = merged_df_away_2025.dropna(subset=['away_player'])


In [27]:
merged_df_home_2025

,home_player,home_H,away_team,Rank,games_played
0,Ben Rice,1.0,NYM,7.0,1
1,Aaron Judge,0.0,NYM,7.0,1
2,Cody Bellinger,0.0,NYM,7.0,1
3,Paul Goldschmidt,0.0,NYM,7.0,1
4,Jasson Dominguez,1.0,NYM,7.0,1
...,...,...,...,...,...
7607,Andy Pages,0.0,ATH,28.0,1
7608,Ryan O'Hearn,1.0,ATH,28.0,1
7609,Michael Conforto,3.0,ATH,28.0,1
7610,Miguel Rojas,0.0,ATH,28.0,1


In [28]:
#renaming the columns of the away players 
merged_df_away_2024 = merged_df_away_2024.rename(columns={
    'away_player': 'player',
    'home_team': 'team',
    'away_H': 'hits',
    'Rank': 'def_rank'
})

In [29]:
#renaming the columns of the away players 
merged_df_home_2024 = merged_df_home_2024.rename(columns={
    'home_player': 'player',
    'away_team': 'team',
    'home_H': 'hits',
    'Rank': 'def_rank'
})

In [30]:
#renaming the columns of the away players 
merged_df_away_2025 = merged_df_away_2025.rename(columns={
    'away_player': 'player',
    'home_team': 'team',
    'away_H': 'hits',
    'Rank': 'def_rank'
})

In [31]:
#renaming the columns of the away players 
merged_df_home_2025 = merged_df_home_2025.rename(columns={
    'home_player': 'player',
    'away_team': 'team',
    'home_H': 'hits',
    'Rank': 'def_rank'
})

In [32]:
# Convert 'hits' to numeric (errors='coerce' turns bad values into NaN)
merged_df_away_2024['hits'] = pd.to_numeric(merged_df_away_2024['hits'], errors='coerce')
merged_df_home_2024['hits'] = pd.to_numeric(merged_df_home_2024['hits'], errors='coerce')
merged_df_away_2025['hits'] = pd.to_numeric(merged_df_away_2025['hits'], errors='coerce')
merged_df_home_2025['hits'] = pd.to_numeric(merged_df_home_2025['hits'], errors='coerce')

In [33]:
#putting 1s in to indicate if they got 1 or 2 hits based off the column
merged_df_away_2024['1_hits_count'] = (merged_df_away_2024['hits'] > 0).astype(int)
merged_df_away_2024['2_hits_count'] = (merged_df_away_2024['hits'] > 1.5).astype(int)
merged_df_away_2024['3_hits_count'] = (merged_df_away_2024['hits'] > 2.5).astype(int)
merged_df_away_2024['4_hits_count'] = (merged_df_away_2024['hits'] > 3.5).astype(int)

merged_df_home_2024['1_hits_count'] = (merged_df_home_2024['hits'] > 0).astype(int)
merged_df_home_2024['2_hits_count'] = (merged_df_home_2024['hits'] > 1.5).astype(int)
merged_df_home_2024['3_hits_count'] = (merged_df_home_2024['hits'] > 2.5).astype(int)
merged_df_home_2024['4_hits_count'] = (merged_df_home_2024['hits'] > 3.5).astype(int)


merged_df_away_2025['1_hits_count'] = (merged_df_away_2025['hits'] > 0).astype(int)
merged_df_away_2025['2_hits_count'] = (merged_df_away_2025['hits'] > 1.5).astype(int)
merged_df_away_2025['3_hits_count'] = (merged_df_away_2025['hits'] > 2.5).astype(int)
merged_df_away_2025['4_hits_count'] = (merged_df_away_2025['hits'] > 3.5).astype(int)


merged_df_home_2025['1_hits_count'] = (merged_df_home_2025['hits'] > 0).astype(int)
merged_df_home_2025['2_hits_count'] = (merged_df_home_2025['hits'] > 1.5).astype(int)
merged_df_home_2025['3_hits_count'] = (merged_df_home_2025['hits'] > 2.5).astype(int)
merged_df_home_2025['4_hits_count'] = (merged_df_home_2025['hits'] > 3.5).astype(int)

In [34]:
merged_df_away_2024

,player,hits,team,def_rank,games_played,1_hits_count,2_hits_count,3_hits_count,4_hits_count
0,Francisco Lindor,0.0,NYY,6.0,1,0,0,0,0
1,Juan Soto,1.0,NYY,6.0,1,1,0,0,0
2,Pete Alonso,1.0,CHC,14.0,1,1,0,0,0
3,Brandon Nimmo,0.0,STL,23.0,1,0,0,0,0
4,Mark Vientos,0.0,NYY,6.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
14257,MJ Melendez,0.0,ATL,8.0,1,0,0,0,0
14258,Hunter Renfroe,1.0,ATL,8.0,1,1,0,0,0
14259,Freddy Fermin,1.0,ATL,8.0,1,1,0,0,0
14260,Maikel Garcia,0.0,ATL,8.0,1,0,0,0,0


In [35]:
#merging one row from df and then the next row from df2 and back and fourth
# Make sure both DataFrames have the same columns

merged_df_away_2024 = merged_df_away_2024.reset_index(drop=True)
merged_df_home_2024 = merged_df_home_2024.reset_index(drop=True)
merged_df_away_2025 = merged_df_away_2025.reset_index(drop=True)
merged_df_home_2025 = merged_df_home_2025.reset_index(drop=True)


# Stack them row by row
interleaved_2024 = pd.concat([merged_df_away_2024, merged_df_home_2024]).sort_index(kind='merge').reset_index(drop=True)
interleaved_2025 = pd.concat([merged_df_away_2025, merged_df_home_2025]).sort_index(kind='merge').reset_index(drop=True)

In [36]:
#unders are only against top 15 teams and overs are only against bot 15 teams, so we split then to get the overs and unders df
unders_2024 = interleaved_2024[interleaved_2024['def_rank'] < 16].reset_index(drop=True)
overs_2024 = interleaved_2024[interleaved_2024['def_rank'] > 15].reset_index(drop=True)
unders_2025 = interleaved_2025[interleaved_2025['def_rank'] < 16].reset_index(drop=True)
overs_2025 = interleaved_2025[interleaved_2025['def_rank'] > 15].reset_index(drop=True)

In [37]:
unders_2024

,player,hits,team,def_rank,games_played,1_hits_count,2_hits_count,3_hits_count,4_hits_count
0,Francisco Lindor,0.0,NYY,6.0,1,0,0,0,0
1,Ben Rice,1.0,NYM,3.0,1,1,0,0,0
2,Juan Soto,1.0,NYY,6.0,1,1,0,0,0
3,Aaron Judge,0.0,NYY,6.0,1,0,0,0,0
4,Pete Alonso,1.0,CHC,14.0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...
12667,MJ Melendez,0.0,ATL,8.0,1,0,0,0,0
12668,Hunter Renfroe,1.0,ATL,8.0,1,1,0,0,0
12669,Freddy Fermin,1.0,ATL,8.0,1,1,0,0,0
12670,Maikel Garcia,0.0,ATL,8.0,1,0,0,0,0


In [38]:
#Staacking unders on top of each other
unders_freq = pd.concat(
    [unders_2024, unders_2025],
    axis=0,
    ignore_index=True
)

In [39]:
unders_freq.to_csv('underfreq.csv')

In [40]:
#stacking overs on top of each other
overs_freq = pd.concat(
    [overs_2024, overs_2025],
    axis=0,
    ignore_index=True
)

In [41]:
#making a new df that sums the 1_hit_count and 2_hit_count and games_played and then creates the season frequency for overs
overs_freq = overs_freq.groupby('player', as_index=False).agg({
    '1_hits_count': 'sum',
    '2_hits_count': 'sum',
    '3_hits_count': 'sum',
    '4_hits_count': 'sum',
    'games_played': 'sum'
})



In [42]:
overs_freq.to_csv('freq.csv')

In [43]:
# Add a column for hit rate aka freq
overs_freq['1_hits_rate'] = overs_freq['1_hits_count'] / overs_freq['games_played']
overs_freq['2_hits_rate'] = overs_freq['2_hits_count'] / overs_freq['games_played']
overs_freq['3_hits_rate'] = overs_freq['3_hits_count'] / overs_freq['games_played']
overs_freq['4_hits_rate'] = overs_freq['4_hits_count'] / overs_freq['games_played']

In [44]:
#making a new df that sums the 1_hit_count and 2_hit_count and games_played and then creates the season frequency for unders 
unders_freq = unders_freq.groupby('player', as_index=False).agg({
    '1_hits_count': 'sum',
    '2_hits_count': 'sum',
    '3_hits_count': 'sum',
    '4_hits_count': 'sum',
    'games_played': 'sum'
})



In [45]:
# Add a column for hit rate
unders_freq['1_hits_rate'] = unders_freq['1_hits_count'] / unders_freq['games_played']
unders_freq['2_hits_rate'] = unders_freq['2_hits_count'] / unders_freq['games_played']
unders_freq['3_hits_rate'] = unders_freq['3_hits_count'] / unders_freq['games_played']
unders_freq['4_hits_rate'] = unders_freq['4_hits_count'] / unders_freq['games_played']

In [46]:
#checking if distrobition is correct, merging them on name to make sure they average less hits against better pitchers and vise versa
final_freq = overs_freq.merge(
    unders_freq,
    on='player',
    how='inner',
    suffixes=('_over', '_under')
)

In [47]:
#dropping all rows that do not meet the creteria: avg less hits against better teams, and more against worse teams
final_freq = final_freq[final_freq['1_hits_rate_under'] <= final_freq['1_hits_rate_over']].reset_index(drop=True)

In [48]:
final_freq

,player,1_hits_count_over,2_hits_count_over,3_hits_count_over,4_hits_count_over,games_played_over,1_hits_rate_over,2_hits_rate_over,3_hits_rate_over,4_hits_rate_over,1_hits_count_under,2_hits_count_under,3_hits_count_under,4_hits_count_under,games_played_under,1_hits_rate_under,2_hits_rate_under,3_hits_rate_under,4_hits_rate_under
0,A.J. Puk,0,0,0,0,1,0.000000,0.000000,0.000000,0.000000,0,0,0,0,2,0.000000,0.000000,0.000000,0.00000
1,Aaron Judge,42,25,9,4,54,0.777778,0.462963,0.166667,0.074074,42,19,3,0,62,0.677419,0.306452,0.048387,0.00000
2,Aaron Schunk,7,6,0,0,14,0.500000,0.428571,0.000000,0.000000,10,2,0,0,26,0.384615,0.076923,0.000000,0.00000
3,Adam Frazier,24,4,1,0,48,0.500000,0.083333,0.020833,0.000000,13,3,0,0,35,0.371429,0.085714,0.000000,0.00000
4,Addison Barger,24,6,1,0,41,0.585366,0.146341,0.024390,0.000000,11,4,0,0,36,0.305556,0.111111,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,Yordan Alvarez,35,16,5,2,49,0.714286,0.326531,0.102041,0.040816,24,9,1,1,42,0.571429,0.214286,0.023810,0.02381
380,Yuli Gurriel,8,2,0,0,16,0.500000,0.125000,0.000000,0.000000,5,0,0,0,15,0.333333,0.000000,0.000000,0.00000
381,Zach DeLoach,5,1,0,0,5,1.000000,0.200000,0.000000,0.000000,3,1,0,0,7,0.428571,0.142857,0.000000,0.00000
382,Zach Dezenzo,10,5,1,0,17,0.588235,0.294118,0.058824,0.000000,13,3,0,0,25,0.520000,0.120000,0.000000,0.00000


In [49]:
#dropping all players that do not have at least 10 games played in this category
final_freq = final_freq[
    (final_freq['games_played_over']  > 30) &
    (final_freq['games_played_under'] > 30)
].reset_index(drop=True)


In [50]:
final_freq.to_csv('asdfasdfasdffasdf.csv')

In [51]:
#making the final dataframes before sending them to the final script with odds
final_over_df=final_freq[['player','1_hits_rate_over','2_hits_rate_over','3_hits_rate_over','4_hits_rate_over']]
final_under_df=final_freq[['player','1_hits_rate_under','2_hits_rate_under','3_hits_rate_under','4_hits_rate_under']]

In [52]:
final_over_df

,player,1_hits_rate_over,2_hits_rate_over,3_hits_rate_over,4_hits_rate_over
0,Aaron Judge,0.777778,0.462963,0.166667,0.074074
1,Adam Frazier,0.500000,0.083333,0.020833,0.000000
2,Addison Barger,0.585366,0.146341,0.024390,0.000000
3,Adley Rutschman,0.600000,0.133333,0.022222,0.000000
4,Adolis Garcia,0.594203,0.217391,0.072464,0.000000
...,...,...,...,...,...
157,Wyatt Langford,0.701754,0.210526,0.087719,0.000000
158,Xavier Edwards,0.800000,0.400000,0.050000,0.025000
159,Yandy Diaz,0.763441,0.387097,0.161290,0.010753
160,Yordan Alvarez,0.714286,0.326531,0.102041,0.040816


In [53]:
final_under_df

,player,1_hits_rate_under,2_hits_rate_under,3_hits_rate_under,4_hits_rate_under
0,Aaron Judge,0.677419,0.306452,0.048387,0.000000
1,Adam Frazier,0.371429,0.085714,0.000000,0.000000
2,Addison Barger,0.305556,0.111111,0.000000,0.000000
3,Adley Rutschman,0.462963,0.148148,0.018519,0.000000
4,Adolis Garcia,0.466667,0.111111,0.044444,0.000000
...,...,...,...,...,...
157,Wyatt Langford,0.565217,0.173913,0.086957,0.021739
158,Xavier Edwards,0.745763,0.322034,0.033898,0.033898
159,Yandy Diaz,0.648649,0.225225,0.036036,0.000000
160,Yordan Alvarez,0.571429,0.214286,0.023810,0.023810


In [54]:
final_over_df.to_csv('overnewhits.csv')

In [55]:
final_under_df.to_csv('undernewhits.csv')